In [2]:
from IO.config import get_usr_config
from IO.result import DetectionResult
from core import detector
from core import freq_transform
from core import signal_generator
from core import utils
from core.utils import MICROSECONDS_IN_SECONDS
from IO import config

import matplotlib.pyplot as plt
import numpy as np
import torch

from datetime import datetime

In [3]:
def unisign_quant(data, n_bits, clip):
    data = torch.Tensor(data)
    w_c = data.clamp(-clip, clip)
    b = torch.pow(torch.tensor(2.0), 1 - n_bits)
    w_q = clip * torch.min(b * torch.round(w_c / (b * clip)), 1 - b)

    return w_q.numpy()


In [24]:
for noise_level in range(-30,11):
    for b in range(2, 11):
        configurations = config.parse_config('../yaml/example.yaml')

        noise_level_db = noise_level
        noise_level_linear = utils.db_to_linear(noise_level_db)

        configurations.signal.num_pos_decision = 1000
        configurations.noise.init_args.top = noise_level_linear
        configurations.noise.init_args.steady_state = noise_level_linear

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_with_noise, _ = generator.get()

        configurations.noise.init_args.top = 0
        configurations.noise.init_args.steady_state = 0

        generator = signal_generator.InputSignalGenerator(configurations.signal, configurations.noise)
        signal_without_noise, _ = generator.get()

        noise_sample = signal_with_noise - signal_without_noise

        Nd = configurations.signal.num_pos_decision
        N = configurations.signal.block_size
        signal_with_noise = signal_with_noise[0:Nd,:,:]
        signal_without_noise = signal_without_noise[0:Nd,:,:]
        noise_sample = noise_sample[0:Nd,:,:]

        coeff_jdht = np.zeros((Nd,N,N))
        for i in range(Nd):
            coeff_jdht[i, :, :] = freq_transform.dht_coeff_jitter(N)/N

        var_jdht_coeff = np.var(coeff_jdht)
        signal_power = np.var(signal_without_noise)

        Bw = b
        w_max = np.abs(coeff_jdht).max()
        qcoeff_jdht = unisign_quant(coeff_jdht, Bw, w_max)
        deltaw = utils.get_delta(Bw, w_max)

        Bx = b
        sigal_max = np.abs(signal_with_noise).max()
        qsignal_with_noise =  unisign_quant(signal_with_noise, Bx, sigal_max)
        deltax = utils.get_delta(Bx, sigal_max)

        result_noise = np.zeros(signal_without_noise.shape)
        result_signal_without_noise = np.zeros(signal_without_noise.shape)
        result_signal_with_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_without_noise = np.zeros(signal_without_noise.shape)
        qresult_signal_with_noise = np.zeros(signal_without_noise.shape)

        start = datetime.now()
        for i in range(Nd):
            result_signal_without_noise[i,:,:] = signal_without_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_noise[i,:,:]  = noise_sample[i,:,:].dot(coeff_jdht[i,:,:].T)
            result_signal_with_noise[i, : ,:] = signal_with_noise[i,:,:].dot(coeff_jdht[i,:,:].T)
            
            qresult_signal_with_noise[i, :, :] = qsignal_with_noise[i,:,:].dot(qcoeff_jdht[i,:,:].T)

        Ba = b
        result_max = result_signal_with_noise.max()
        deltay = utils.get_delta(Ba, result_max)
        qresult_signal_with_noise = unisign_quant(qresult_signal_with_noise, Ba, result_max)
        end = datetime.now()

        total_dp_noise = qresult_signal_with_noise - result_signal_without_noise
        var_total_dp_noise = total_dp_noise[:,:,3].var()
        A = deltay**2/12
        B = N * var_jdht_coeff * noise_level_linear
        C = N/12 * var_jdht_coeff * deltax**2
        D = N/12 * signal_without_noise.var() * deltaw**2
        E = N/12 * noise_level_linear * deltaw**2

        print('total time (s):', utils.format_float((end-start).total_seconds()), 'B:', b,
              noise_level, 'dB:', 10*np.log10((A+B+C+D+E)/var_total_dp_noise))

total time (s):  0.008 B: 2 -30 dB: -0.6295319422019183
total time (s):  0.007 B: 3 -30 dB: -0.9539575853979847
total time (s):  0.009 B: 4 -30 dB: -0.1877987981631679
total time (s):  0.008 B: 5 -30 dB: -0.12078036051362663
total time (s):  0.009 B: 6 -30 dB: 0.13128010253289543
total time (s):  0.008 B: 7 -30 dB: -0.396270738632688
total time (s):  0.008 B: 8 -30 dB: -0.20147363779604768
total time (s):  0.009 B: 9 -30 dB: 0.010562523209581377
total time (s):  0.008 B: 10 -30 dB: 0.2227164168374251
total time (s):  0.007 B: 2 -29 dB: -0.6175245586407728
total time (s):  0.007 B: 3 -29 dB: -1.0460993830913623
total time (s):  0.007 B: 4 -29 dB: -0.06476762446059636
total time (s):  0.007 B: 5 -29 dB: -0.5538611126294235
total time (s):  0.007 B: 6 -29 dB: 0.1198296590636917
total time (s):  0.007 B: 7 -29 dB: -0.0631486396310989
total time (s):  0.007 B: 8 -29 dB: 0.06889299675674644
total time (s):  0.008 B: 9 -29 dB: -0.33131242551448736
total time (s):  0.008 B: 10 -29 dB: 0.126802

total time (s):  0.007 B: 4 -14 dB: 0.18941902428462556
total time (s):  0.009 B: 5 -14 dB: 0.321190996562026
total time (s):  0.009 B: 6 -14 dB: 0.3420119711928281
total time (s):  0.008 B: 7 -14 dB: -0.12897282868367355
total time (s):  0.009 B: 8 -14 dB: -0.14735177764349144
total time (s):  0.008 B: 9 -14 dB: 0.0643141215989906
total time (s):  0.009 B: 10 -14 dB: 0.05168453871614846
total time (s):  0.008 B: 2 -13 dB: 0.7509641310595684
total time (s):  0.009 B: 3 -13 dB: 0.04441994062630802
total time (s):  0.008 B: 4 -13 dB: 0.008510739988664988
total time (s):  0.009 B: 5 -13 dB: -0.21331019657683647
total time (s):  0.009 B: 6 -13 dB: 0.35932708890975296
total time (s):  0.009 B: 7 -13 dB: 0.24834316014108299
total time (s):  0.008 B: 8 -13 dB: -0.05876479273153052
total time (s):  0.008 B: 9 -13 dB: -0.0342545116080102
total time (s):  0.008 B: 10 -13 dB: -0.1592600940185796
total time (s):  0.008 B: 2 -12 dB: 0.28342401556200747
total time (s):  0.007 B: 3 -12 dB: -0.0684538

total time (s):  0.009 B: 9 2 dB: 0.16962476412838431
total time (s):  0.008 B: 10 2 dB: -0.08687706401231891
total time (s):  0.007 B: 2 3 dB: 1.4808218051575923
total time (s):  0.007 B: 3 3 dB: 0.47192497164296054
total time (s):  0.009 B: 4 3 dB: 0.3215388313897615
total time (s):  0.008 B: 5 3 dB: 0.08582703722839594
total time (s):  0.008 B: 6 3 dB: 0.11068075571628137
total time (s):  0.008 B: 7 3 dB: 0.025744104297658953
total time (s):  0.009 B: 8 3 dB: 0.08833585112255044
total time (s):  0.007 B: 9 3 dB: 0.19581458187622675
total time (s):  0.008 B: 10 3 dB: -0.06136380007068944
total time (s):  0.008 B: 2 4 dB: 2.6790489578687398
total time (s):  0.008 B: 3 4 dB: 0.4742055841910182
total time (s):  0.007 B: 4 4 dB: 0.5787281115735394
total time (s):  0.008 B: 5 4 dB: -0.06394471763136861
total time (s):  0.010 B: 6 4 dB: -0.007648062127575923
total time (s):  0.008 B: 7 4 dB: -0.09092215021676021
total time (s):  0.009 B: 8 4 dB: 0.024556742482695973
total time (s):  0.008 